In [1]:
'''this notebook heavily refers to the post https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/'''

'this notebook heavily refers to the post https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/'

In [2]:
import nltk
#nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#for Stemming
from nltk.stem.snowball import DanishStemmer

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\nmf.py:972: DeprecationWarning: invalid escape sequence \s
  """
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\nmf.py:1186: DeprecationWarning: invalid escape sequence \s
  """
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\manifold\t_sne.py:420: DeprecationWarning: invalid escape sequence \s
  """


In [3]:
import os,sys
parentPath = os.path.join(os.path.dirname(os.getcwd()),"tools")
sys.path.insert(0, parentPath)
from scripts_py import preprocessing as pre

#### Prepare stopwords

In [4]:
# NLTK Stop words 
# TODO change to our own stop words list
from nltk.corpus import stopwords
stop_words = stopwords.words('danish')
stop_words.extend(['paa','ved','saa','den'])

In [5]:
import codecs
def creat_file_list(author_name):
    file_list=['non']
    cwd=os.getcwd()
    parent_path = os.path.dirname(cwd)
    des_pos=os.path.join(parent_path,"ADL","metadata","metadata_adl.csv")
    fh=codecs.open(des_pos,'r','utf-8')
    for line in fh.readlines():
        L=line.split(",")
        if(re.search(author_name,L[1])):
            temp_string=L[0]+".txt"
            file_list.append(temp_string)
    return file_list[1:]

#### Select author here

In [ ]:
author = "Grundtvig"
articles = creat_file_list(author)

#### Read in articles

In [7]:
data = list()
"""add any articles as you like if you want to analyze specific articles"""
#articles = ["aare01val_workid69870.txt","aakjaer01val_workid55565.txt","aakjaer01val_workid55662.txt"
#            ,"aakjaer01val_workid55881.txt"]
"""following line process all the articles but this may require huge computing power"""
#articles=os.listdir(os.path.join(os.path.dirname(os.getcwd()),"ADL","plain"))
for ele in articles:
    article = os.path.join(os.path.dirname(os.getcwd()),"ADL","plain",ele)
    with open(article, encoding ="utf-8") as f:
        lines = f.readlines()
    data.append( ' '.join(lines))

#### Preprocessing

In [8]:
nalpha_content = [pre.re_nalpha(ele) for ele in data]
tokenized_content = [pre.tokenizer(ele) for ele in nalpha_content]

In [10]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def stemmer(wordList):
    """stemming the words for later processing"""
    stemmer = DanishStemmer()
    return [stemmer.stem(tempWord) for tempWord in wordList]

In [11]:
data_words_nostops=remove_stopwords(tokenized_content)

In [12]:
bigram = gensim.models.Phrases(tokenized_content, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
#trigram = gensim.models.Phrases(bigram[tokenized_content], threshold=100)
#trigram_mod = gensim.models.phrases.Phraser(trigram)

In [13]:

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
data_stemmed = [stemmer(ele) for ele in data_words_bigrams]
data_stemmed=remove_stopwords(data_stemmed)

In [14]:
# Create Dictionary
id2word = corpora.Dictionary(data_stemmed)
# Create Corpus
texts = data_stemmed

In [15]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [17]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [18]:
%store lda_model

Stored 'lda_model' (LdaModel)


In [19]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


[(0,
  '0.019*"gud" + 0.011*"liv" + 0.011*"ei" + 0.010*"aand" + 0.009*"dit" + '
  '0.009*"død" + 0.007*"kun" + 0.007*"ord" + 0.006*"verd" + 0.006*"evig"'),
 (1,
  '0.013*"kan" + 0.010*"aand" + 0.010*"kirk" + 0.009*"kun" + 0.008*"maa" + '
  '0.008*"naar" + 0.007*"ord" + 0.007*"hel" + 0.007*"tro" + 0.006*"gud"'),
 (2,
  '0.007*"kong" + 0.007*"kun" + 0.007*"stor" + 0.006*"hel" + 0.006*"rom" + '
  '0.006*"kom" + 0.005*"ogsaa" + 0.005*"vel" + 0.005*"baad" + 0.005*"vild"'),
 (3,
  '0.013*"kong" + 0.010*"ei" + 0.005*"kom" + 0.005*"dannemark" + 0.003*"kun" + '
  '0.003*"gaml" + 0.003*"søn" + 0.003*"ord" + 0.003*"sagd" + 0.003*"dag"'),
 (4,
  '0.001*"jordensaa" + 0.001*"rethan" + 0.001*"tidsom" + 0.001*"givetat" + '
  '0.001*"tilbage" + 0.001*"sønjeg" + 0.001*"svigei" + 0.001*"saamed" + '
  '0.001*"letsom" + 0.001*"matthisonhans"'),
 (5,
  '0.017*"ei" + 0.011*"gud" + 0.007*"dit" + 0.006*"kun" + 0.005*"mon" + '
  '0.004*"kan" + 0.004*"ord" + 0.004*"gaml" + 0.003*"kirk" + 0.003*"maa"'),
 (6,
  '0

In [20]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_stemmed, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.74795057978

Coherence Score:  0.459748185629


In [21]:

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
11     38.210047        1       1 -0.352766 -0.104252
1      18.816450        1       2 -0.314486 -0.133787
2      16.645257        1       3 -0.271210 -0.154307
18      4.235867        1       4 -0.159241  0.052593
14      3.653469        1       5 -0.204848  0.131487
3       3.598546        1       6 -0.139745  0.050382
5       3.558330        1       7 -0.122711  0.150842
0       3.416661        1       8 -0.171760  0.178758
12      1.791450        1       9  0.064693 -0.181214
10      1.775918        1      10  0.094971 -0.072359
8       1.234361        1      11 -0.039278  0.144828
9       1.195572        1      12  0.098784 -0.078577
6       0.600576        1      13  0.185641  0.009599
19      0.343764        1      14  0.187375 -0.003069
17      0.218312        1      15  0.195345 -0.001104
16      0.203050        1      16  0.191869 -0.001334
13      0.193561        1      17  0.193320  0.001998
7       0.145949        1      18  0.188886  0.003117
15      0.083439        1      19  0.188016  0.002722
4       0.079422        1      20  0.187146  0.003676, topic_info=      Category          Freq             Term         Total  loglift  logprob
term                                                                         
10     Default   9434.000000               ei   9434.000000  30.0000  30.0000
1835   Default   2059.000000            dansk   2059.000000  29.0000  29.0000
193    Default   6606.000000              gud   6606.000000  28.0000  28.0000
74     Default   4814.000000              ord   4814.000000  27.0000  27.0000
116    Default   5774.000000             aand   5774.000000  26.0000  26.0000
44     Default  10871.000000              kan  10871.000000  25.0000  25.0000
748    Default   3873.000000              liv   3873.000000  24.0000  24.0000
50     Default   3225.000000             kong   3225.000000  23.0000  23.0000
52     Default   9854.000000              kun   9854.000000  22.0000  22.0000
8      Default   3270.000000              død   3270.000000  21.0000  21.0000
397    Default   2415.000000            hjert   2415.000000  20.0000  20.0000
149    Default   1428.000000          danmark   1428.000000  19.0000  19.0000
108    Default   2927.000000             verd   2927.000000  18.0000  18.0000
256    Default   1810.000000             nord   1810.000000  17.0000  17.0000
1070   Default   3125.000000             kirk   3125.000000  16.0000  16.0000
22143  Default    736.000000              dam    736.000000  15.0000  15.0000
3673   Default    921.000000            krist    921.000000  14.0000  14.0000
0      Default   2483.000000               al   2483.000000  13.0000  13.0000
384    Default   2590.000000             giør   2590.000000  12.0000  12.0000
901    Default   6005.000000              maa   6005.000000  11.0000  11.0000
146    Default   3230.000000              dag   3230.000000  10.0000  10.0000
409    Default   5233.000000              kom   5233.000000   9.0000   9.0000
60     Default   1810.000000              lys   1810.000000   8.0000   8.0000
561    Default   2510.000000          mennesk   2510.000000   7.0000   7.0000
42     Default   2000.000000             jord   2000.000000   6.0000   6.0000
2723   Default   1016.000000              ere   1016.000000   5.0000   5.0000
445    Default   3373.000000             sand   3373.000000   4.0000   4.0000
102    Default   2471.000000              tro   2471.000000   3.0000   3.0000
154    Default   1170.000000              dit   1170.000000   2.0000   2.0000
13     Default   1334.000000             evig   1334.000000   1.0000   1.0000
...        ...           ...              ...           ...      ...      ...
1783   Topic20      0.750726           saamed      1.366180   6.5394  -7.2283
1777   Topic20      0.720878           letsom      1.336332   6.5209  -7.2689
14825  Topic20      0.718206    matthis

In [22]:
%store vis

Stored 'vis' (PreparedData)


### Model exploration

#### Finding the dominant topic in each sentence

In [23]:
"""the following code will help to determine what topic a given document is about"""
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = row[0]
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        topic_num = row[0][0]
        prop_topic = row[0][1]
        wp = ldamodel.show_topic(topic_num)
        topic_keywords = ", ".join([word for word, prop in wp])
        sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [24]:

df_topic_sents_keywords = format_topics_sentences(lda_model, corpus=corpus, texts=data)

In [25]:

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,0.0,0.7284,"gud, liv, ei, aand, dit, død, kun, ord, verd, ...","No. 196.\n *)Jeg som et Barn mig glæde vil,Og ..."
1,1,7.0,0.3263,"herod, delphi, melodi, slagterbænk, guldsom, d...",HENRICH STEFFENS.VED Henrik Steffens' Død den ...
2,2,2.0,0.3682,"kong, kun, stor, hel, rom, kom, ogsaa, vel, ba...","No. 40\n .Verden er, som Man den tager,Herlig ..."
3,3,14.0,0.9286,"gud, ord, ei, hjert, kun, verd, aand, lys, dag...",No. 63\n . *)Paradis-Gaard og Folke-Vang!Synge...
4,4,8.0,0.5302,"gud, død, ei, kan, al, no, liv, ord, aand, vort","No. 327\n .Gud-Fader udsendte sit Naadens-Ord,..."
5,5,14.0,0.7348,"gud, ord, ei, hjert, kun, verd, aand, lys, dag...","No. 200\n .Guds Engle sang i Stjerne-Tal,Stjer..."
6,6,14.0,0.2906,"gud, ord, ei, hjert, kun, verd, aand, lys, dag...",No. 156\n . *)Stjernernes Skaber og Himmelens ...
7,7,10.0,0.3625,"dam, odin, thor, myth, asern, jet, lok, nord, ...","No. 119\n .Syng, Guds Folk! med Hjertens Glæde..."
8,8,17.0,0.7656,"glemmes_aldr, kvædl, willemo, syrith, himlensa...","No. 2\n .Adam og Eva\n .Ordet sig hviled,Engle..."
9,9,8.0,0.9025,"gud, død, ei, kan, al, no, liv, ord, aand, vort",No\n . 86.*)Mel.\n : Et Barn er født i Bethehe...


#### Find the most representative document for each topic

In [26]:
"""find the documents a given topic has contributed to the most and infer the topic by reading that document."""
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.9327,"gud, liv, ei, aand, dit, død, kun, ord, verd, ...",No. 354.\n *)Sin Eenbaarne Gud hengivetHar for...
1,1.0,0.9229,"kan, aand, kirk, kun, maa, naar, ord, hel, tro...",DEN CHRISTELIGEBØRNELÆRDOM.DEN Række Afhandlin...
2,2.0,0.7049,"kong, kun, stor, hel, rom, kom, ogsaa, vel, ba...",HAANDBOG I VERDENSHISTORIEN. ANDEN DEEL.\n MID...
3,3.0,0.6854,"kong, ei, kom, dannemark, kun, gaml, søn, ord,...","No. 211\n . *)Med sin Alabaster-Krukke,Fuld af..."
4,4.0,0.5208,"jordensaa, rethan, tidsom, givetat, tilbage, s...","No. 38\n .Christne, med Skiel,Kaldes de vel,Al..."


#### Topic distribution across documents

In [27]:
"""understand the volume and distribution of topics in order to judge how widely it was discussed"""

'understand the volume and distribution of topics in order to judge how widely it was discussed'

In [28]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0,0.0,"gud, liv, ei, aand, dit, død, kun, ord, verd, ...",242.0,0.3482
1,7.0,"herod, delphi, melodi, slagterbænk, guldsom, d...",30.0,0.0432
2,2.0,"kong, kun, stor, hel, rom, kom, ogsaa, vel, ba...",6.0,0.0086
3,14.0,"gud, ord, ei, hjert, kun, verd, aand, lys, dag...",13.0,0.0187
4,8.0,"gud, død, ei, kan, al, no, liv, ord, aand, vort",10.0,0.0144
5,14.0,"gud, ord, ei, hjert, kun, verd, aand, lys, dag...",50.0,0.0719
6,14.0,"gud, ord, ei, hjert, kun, verd, aand, lys, dag...",13.0,0.0187
7,10.0,"dam, odin, thor, myth, asern, jet, lok, nord, ...",11.0,0.0158
8,17.0,"glemmes_aldr, kvædl, willemo, syrith, himlensa...",92.0,0.1324
9,8.0,"gud, død, ei, kan, al, no, liv, ord, aand, vort",3.0,0.0043
